In [1]:
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install datasets wandb huggingface_hub pandas
!pip install -U sentence-transformers
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.0/419.0 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Load dataset and encode labels

In [27]:
from datasets import load_dataset, Dataset, DatasetDict

train_dataset = load_dataset("Muhammad2003/routing-dataset", split="train")
train_dataset = train_dataset.rename_column("question", "sentence")

In [28]:
train_dataset.set_format('pandas')
df = train_dataset[:14271]
df['label']=df['label'].astype('category').cat.codes
train_dataset = Dataset.from_pandas(df)
print(train_dataset)

Dataset({
    features: ['sentence', 'label'],
    num_rows: 14271
})


## Load `BAAI/bge-small-en-v1.5` embedding model and `BatchAllTripletLoss`
Loss functions mainly depend on dataset structure, a detailed guide is given on [SentenceTransformers Documentation](https://www.sbert.net/docs/sentence_transformer/loss_overview.html)

In [3]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.losses import BatchAllTripletLoss

# Load a model to train/finetune
model = SentenceTransformer("BAAI/bge-small-en-v1.5")

# Initialize triplet loss
loss = BatchAllTripletLoss(model)


## Intialize WandB

In [4]:
import wandb

#monitering login
wandb.login(key="")
run = wandb.init(project='embed', job_type="training", anonymous="allow")

wandb: Currently logged in as: muhammadbin-2003 (rethinkai). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Setup Training Arguments and start training

In [33]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments, BatchSamplers

args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/router-embedding",
    # Optional training parameters:
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    # eval_strategy="steps",
    # eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name="bge-triplet",  # Will be used in W&B if `wandb` is installed
)

In [34]:
from sentence_transformers import SentenceTransformerTrainer
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    loss=loss,
    evaluator=None
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [35]:
trainer.train()

Step,Training Loss
100,4.499200
200,4.665300
300,4.473800
400,4.433600
500,4.355800
600,4.430600
700,4.538400
800,4.428600
900,4.338800
1000,4.365100


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=1784, training_loss=4.422706672428969, metrics={'train_runtime': 132.4336, 'train_samples_per_second': 215.519, 'train_steps_per_second': 13.471, 'total_flos': 0.0, 'train_loss': 4.422706672428969, 'epoch': 2.0})

## Save the model and push to Hub

In [36]:
model.save_pretrained("models/router-embedding")

In [37]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [40]:
!huggingface-cli login --token ''

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [41]:
model.push_to_hub("Muhammad2003/router-embedding")

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

'https://huggingface.co/Muhammad2003/router-embedding/commit/d08bd7d066d5450ce0311151cbfea6bbea7f2230'

In [42]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("Muhammad2003/router-embedding")
# Run inference
sentences = [
    'How can the person ensure they receive the necessary compensation for their work-related injury?',
    'Is there a law in Oklahoma that restricts the distance of a dispensary to a baseball field?',
    'Considering the complexities of property rights, due process, and public safety, what are the ethical and legal considerations surrounding citizens taking possession of unattended animals in public areas, and how do these actions intersect with constitutional rights and property laws?',
]
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [3, 3]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

(3, 384)
torch.Size([3, 3])


In [43]:
embeddings

array([[-0.01034122, -0.11917762,  0.04457978, ...,  0.09624283,
        -0.0318978 , -0.03279097],
       [-0.00842274, -0.11669022,  0.04836053, ...,  0.10686176,
        -0.02774303, -0.03649817],
       [ 0.00247131,  0.11810412, -0.03308529, ..., -0.11345071,
         0.03245148,  0.03161241]], dtype=float32)